In [1]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
import time

mesh = ngs.Mesh('whatever.vol')
mesh.Curve(2)


In [2]:
fesVectorPotential = ngs.HCurl(mesh, dirichlet = "coil_outer|coil_inner|coil_up|coil_down", definedon='coil')
TotalCurrent = 1000
coil_depth = 0.1
    
normal = ngs.specialcf.normal(mesh.dim)
gfuDirichlet = ngs.GridFunction(fesVectorPotential)
g = ngs.CF([(0,0,TotalCurrent/coil_depth) if bc=="coil_inner" else (0,0,0) for bc in mesh.GetBoundaries()])
gfuDirichlet.Set(g,ngs.BND)

# Draw(gfuDirichlet, mesh, vectors = { "grid_size":200}, draw_vol=False, clipping={"z": -1, "dist": 0.064});

In [3]:
from ngsolve.krylovspace import CGSolver

# Weak form
T = fesVectorPotential.TrialFunction()
Tstar = fesVectorPotential.TestFunction()
Kt = ngs.BilinearForm(ngs.curl(Tstar)*ngs.curl(T)*ngs.dx)
    
# Assembly
with ngs.TaskManager():
    Kt.Assemble()

r = - Kt.mat * gfuDirichlet.vec

# Solving
maxres = 1e-8
maxit = 1000
                              
Tsol = ngs.GridFunction(fesVectorPotential)
print("Solving...")

#with TaskManager():
iterativeSolver = CGSolver(Kt.mat, freedofs = fesVectorPotential.FreeDofs(), tol  = maxres,  maxiter = maxit)
Tsol.vec.data = iterativeSolver * r + gfuDirichlet.vec

if len(iterativeSolver.residuals) == maxit: print("... Failure!")
else: print("... Success!")
print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")

Draw(ngs.curl(Tsol), mesh, vectors = { "grid_size" : 150},clipping = {"x" : 0, "y" : 0, "z" : -1, "dist" : 0})

Solving...
... Success!
Number of iterations = 128/1000 | Residual = 0.0036382463494895882


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
# ngs.Integrate(ngs.curl(Tsol), mesh)

n = ngs.specialcf.normal(mesh.dim)
flux_bd = ngs.Integrate(ngs.curl(Tsol) * n, mesh, definedon = mesh.Boundaries("coil_cut_2"))
print(flux_bd)

999.9999999999959


In [8]:
Hcurl = ngs.HCurl(mesh, definedon='coil')
u,v = Hcurl.TnT()

K = ngs.BilinearForm(ngs.curl(u)*ngs.curl(v)*ngs.dx)
f = ngs.


In [6]:
H1 = ngs.H1(mesh, order=1, definedon="coil", dirichlet="coil_cut_1")
phi,psi = H1.TnT()
n = ngs.specialcf.normal(mesh.dim)


sigma = 58.7e6
crosssection = 0.0025

bfa = ngs.BilinearForm(sigma*ngs.grad(phi)*ngs.grad(psi)*ngs.dx).Assemble()
inv = bfa.mat.Inverse(freedofs = H1.FreeDofs(), inverse="sparsecholesky")


lff = ngs.LinearForm(n*ngs.grad(psi).Trace()*ngs.ds("coil_cut")).Assemble()

gfphi = ngs.GridFunction(H1)
gfphi.vec.data = inv * lff.vec

Draw(gfphi, draw_vol=False, clipping={"z": -1, "dist": 0.064});

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [7]:
p_in = ngs.CoefficientFunction( 10.0 )

lff = ngs.LinearForm(p_in*n*ngs.grad(psi)*ngs.ds("coil_cut")).Assemble()
print(min(lff.vec.FV().NumPy()))
print(max(lff.vec.FV().NumPy()))

NgException: Testfunction does not support BND-forms, maybe a Trace() operator is missing